In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from ukb_gwas import *
from scipy import stats

In [2]:
# getting standard snp set for EDUYEARS
eduyears_lead_snps = pd.read_excel('41588_2018_147_MOESM3_ESM.xlsx', 
                                  sheet_name = '2. EduYears Lead SNPs',
                                  skipfooter = 2, header = 1, index_col = None )
snplist = eduyears_lead_snps.SNP.tolist()
print(len(snplist))
print(snplist[:5])

1271
['rs9859556', 'rs7029718', 'rs1334297', 'rs9375188', 'rs2526398']


In [4]:
# save to file
with open('lee_eduyears_snps.txt','w') as o:
    for snp in snplist:
        o.write(snp + '\n')

In [5]:
eduyears = pd.read_csv('../pheno/pheno_eduYears.csv',delimiter = ' ')
eduyears.head(2)

,eid,eid.1,val
0,3609487,3609487,20
1,1194904,1194904,20


In [8]:
pob_por_migration = pd.read_csv('../00_csv_files/pob_por_migration.csv')
pob_por_migration.head(2)

,eid,POB_east,POB_north,POR_east,POR_north,distances,angles,POR_POB_popDensity
0,1136349,68500,801500,252000.0,663000.0,229901.065678,127.044351,389.270739
1,1071275,66500,798500,211000.0,665000.0,196729.509734,132.734083,0.798042


In [10]:
mus_wba_eids = [int(x.split()[0]) for x in open('../00_csv_files/mus_wba_eids.csv','r').readlines()]
print(mus_wba_eids[0])

4194304


In [11]:
all_movers = pob_por_migration[pob_por_migration.distances > 20000].eid.tolist()
all_stayers = pob_por_migration[pob_por_migration.distances <= 20000].eid.tolist()
wba_movers = list(set(all_movers)&set(mus_wba_eids))
wba_stayers = list(set(all_stayers)&set(mus_wba_eids))
print(len(wba_movers),len(wba_stayers))

152227 177519


In [12]:
edvals = np.unique(eduyears.val.tolist())
edval_stayers = {}
edval_movers = {}
for edval in edvals:
    eids_for_edval = eduyears[eduyears.val==edval].eid.tolist()
    movers_val = list(set(wba_movers)&set(eids_for_edval))
    stayers_val = list(set(wba_stayers)&set(eids_for_edval))
    
    edval_stayers[edval] = stayers_val
    edval_movers[edval] = movers_val
    
    # if fewer movers, sample from stayers
    if len(movers_val) < len(stayers_val): 
        edval_numToSample[edval] = len(movers_val)
    # if fewer stayers, sample from movers
    else:
        edval_numToSample[edval] = len(stayers_val)

print(edval_numToSample)

{7: 14220, 10: 33176, 13: 18188, 15: 7873, 19: 7209, 20: 32884}


In [18]:
for val in sorted(edval_stayers.keys()):
    with open('eduyearsvals_' + str(val) + '_stayers.txt','w') as o:
        for eid in edval_stayers[val]:
            o.write(str(eid) + '\n')
    with open('eduyearsvals_' + str(val) + '_movers.txt','w') as o:
        for eid in edval_movers[val]:
            o.write(str(eid) + '\n')

with open('eduyearsvals_counts.txt','w') as o:
    for val in sorted(edval_numToSample.keys()):
        o.write(str(val) + ' ' + str(edval_numToSample[val]) + '\n')

In [ ]:
# copy to cbsu:
# scp eduyearsvals_* igw9@cbsulogin.tc.cornell.edu:/bscb/bscb09/500k_ukb/ian/bootstrap_ea_rarefication/.

In [34]:
# script to save lots of files of rarefied movers and stayers

numFilesToSave = 2

# get numbers to sample
edval_numToSample = {}
with open('eduyearsvals_counts.txt','r') as f:
    for line in f:
        stuff = line.rstrip().split()
        edval_numToSample[int(stuff[0])] = int(stuff[1])

#print(edval_numToSample)

# make dictionaries for movers and stayers
# key = eduyears val; value = list of eids

edval_stayers = {}
edval_movers = {}

for val in sorted(edval_numToSample.keys()):
    sfile = 'eduyearsvals_' + str(val) + '_stayers.txt'
    mfile = 'eduyearsvals_' + str(val) + '_movers.txt'
    edval_stayers[val] = [int(x.rstrip()) for x in open(sfile,'r').readlines()]
    edval_movers[val] =  [int(x.rstrip()) for x in open(mfile,'r').readlines()]
    
#print(len(edval_movers[20]))

#print(edval_stayers[20][:5])

for i in np.arange(numFiles):
    s_outfile = 'rarefied_stayers_' + str(i+1) + '.txt'
    m_outfile = 'rarefied_movers_' + str(i+1) + '.txt'
    s = open(s_outfile,'w')
    m = open(m_outfile,'w')
    
    for val in sorted(edval_numToSample.keys()):
        num_to_get = edval_numToSample[val]
        stayer_samples = np.random.choice(edval_stayers[val],num_to_get,replace=False)
        mover_samples = np.random.choice(edval_movers[val],num_to_get,replace=False)
        
        for eid in stayer_samples:
            s.write(str(eid) + ' ' + str(eid) + '\n')
        for eid in mover_samples:
            m.write(str(eid) + ' ' + str(eid) + '\n')
    
    s.close()
    m.close()

In [38]:
#!/usr/bin/python
import os

# script to run plink on the rarefied samples
s1 = '/home/bscb09/bin/plink2.0 '
s2 = '--bfile chrMerged_lee_snps '
s3 = ' --keep rarefied_' 
# add 'stayers' or 'movers' here
# add _number here
s4 = '.txt --linear hide-covar '
s5 = '--extract lee_eduyears_snps.txt '
s6 = '--covar /bscb/bscb09/500k_ukb/ian/gwas_files/covar_sex_pcs_batch_chip.txt '
s7 = '--covar-variance-standardize '
s8 = '--pheno /bscb/bscb09/500k_ukb/ian/gwas_files/pheno_eduyears.csv '
s9 = '--threads 4 --out plink_'
# add 'stayers' or 'movers' here
# add _number here


for n in np.arange(2):
    for group in ['stayers','movers']:
        i = n+1
        print(s1 + s2 + s3 + group + '_' + str(i) + s4 + s5 + s6 + s7 + s8 + s9 + group + '_' + str(i))


/home/bscb09/bin/plink2.0 --bfile chrMerged_lee_snps  --keep rarefied_stayers_1.txt --linear hide-covar --extract lee_eduyears_snps.txt --covar /bscb/bscb09/500k_ukb/ian/gwas_files/covar_sex_pcs_batch_chip.txt --covar-variance-standardize --pheno /bscb/bscb09/500k_ukb/ian/gwas_files/pheno_eduyears.csv --threads 4 --out plink_stayers_1
/home/bscb09/bin/plink2.0 --bfile chrMerged_lee_snps  --keep rarefied_movers_1.txt --linear hide-covar --extract lee_eduyears_snps.txt --covar /bscb/bscb09/500k_ukb/ian/gwas_files/covar_sex_pcs_batch_chip.txt --covar-variance-standardize --pheno /bscb/bscb09/500k_ukb/ian/gwas_files/pheno_eduyears.csv --threads 4 --out plink_movers_1
/home/bscb09/bin/plink2.0 --bfile chrMerged_lee_snps  --keep rarefied_stayers_2.txt --linear hide-covar --extract lee_eduyears_snps.txt --covar /bscb/bscb09/500k_ukb/ian/gwas_files/covar_sex_pcs_batch_chip.txt --covar-variance-standardize --pheno /bscb/bscb09/500k_ukb/ian/gwas_files/pheno_eduyears.csv --threads 4 --out plink_s

In [ ]:
# collect betas for each run (e.g. run 1)
# process as I've done before (need to think about absolute value)
# report average stayer, average mover